# Data analysis - `scikit-learn`

In [ ]:
%matplotlib inline
import numpy as np
import pandas as pd
from scipy.stats import zscore

from mvpa2.tutorial_suite import fmri_dataset
from mvpa2.base.hdf5 import h5save

/home/line/anaconda3/envs/mvpa/lib/python2.7/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


 * Please note: warnings are printed only once, but underlying problem might occur many times *


/home/line/anaconda3/envs/mvpa/lib/python2.7/site-packages/statsmodels/compat/pandas.py:56: FutureWarning: The pandas.core.datetools module is deprecated and will be removed in a future version. Please use the pandas.tseries module instead.
  from pandas.core import datetools


# Create dataset

In [ ]:
def read_dataset(filename, outlier_thr=5):

    # Read csv file
    df = pd.read_csv(filename)

    # extract relevant variables
    sub_id = df['PAC_ID']
    df = df.drop('PAC_ID', 1)
    header = df.keys()
    
    # Clean dataset - drop subjects with values above `outlier_thr` STD
    outliers = np.sum((np.abs(zscore(df)) > outlier_thr), 1) != 0
    print('%d outliers detected.' % outliers.sum())
    data = np.array(df.drop(np.where(outliers)[0]))
    sub_id = sub_id[np.invert(outliers)]
    
    # zscore data
    data = zscore(data)

    # Reset Gender and Scanner values to nominal values
    data[:,0] = (data[:,0]>0) + 1
    data[:,2] = (data[:,2]>0) + 1
    data[:,4] = [np.where(i==np.unique(data[:,4]))[0][0] + 1for i in data[:,4]]

    return pd.DataFrame(data, columns=header), sub_id

In [ ]:
data, sub_id = read_dataset('data/PAC2018_Covariates_detailed.csv', outlier_thr=5)
data.head()

8 outliers detected.


,Label,Age,Gender,TIV,Scanner,Tvoxels,Tmean,Tmedian,Tstd,Tmax,...,Right_Cerebral_White_Matter,Right_Cerebral_Cortex,Right_Lateral_Ventricle,Right_Thalamus,Right_Caudate,Right_Putamen,Right_Pallidum,Right_Hippocampus,Right_Amygdala,Right_Accumbens
0,1.0,1.610405,1.0,1.479924,2.0,-0.890455,1.091014,1.080367,1.034026,0.060878,...,1.093750,1.094903,1.037705,-0.896948,-0.255150,-0.724690,-0.269052,1.791264,1.591691,0.091529
1,1.0,-1.146076,1.0,-0.052883,1.0,1.134709,-0.118805,-0.147065,0.290905,0.904814,...,-0.060199,-0.013714,0.079673,0.634875,1.097503,0.176912,-0.143525,0.223282,0.283818,-0.110870
2,1.0,-0.200997,2.0,-0.322187,2.0,-0.910314,0.213228,0.466651,-0.390523,0.488056,...,0.167886,0.162364,0.443724,0.107536,-0.337023,-0.458773,-0.199329,0.879846,0.603997,0.182364
3,1.0,-0.200997,1.0,1.526994,3.0,-0.888128,1.178024,1.141739,1.173458,0.863138,...,1.022908,1.000870,2.157792,1.573471,2.835140,1.080137,0.815276,0.185643,0.730864,2.010200
4,1.0,2.004188,2.0,-0.934370,1.0,1.027570,-2.074238,-2.049584,-2.070608,-0.824733,...,-2.148225,-2.165083,-1.193547,-1.593558,-2.270355,-1.290505,-1.337832,-1.919742,-2.127827,-1.856324


# Prepare Dataset for PyMVPA

### Balance datasets (1 for `Scanner 1` and 1 for `Scanner 2/3`)

In [ ]:
scan1_id = data['Scanner'] == 1
labels1 = np.array(data[scan1_id]['Label'])
sub_id1 = np.array(sub_id)[scan1_id]

scan23_id = data['Scanner'] != 1
labels23 = np.array(data[scan23_id]['Label'])
sub_id23 = np.array(sub_id)[scan23_id]

In [ ]:
def balance_dataset(sub_id, labels):
    max_label_size = np.min([np.sum(lab == labels) 
                             for lab in np.unique(labels)])

    labels_1 = np.where(labels == 1)[0]
    np.random.shuffle(labels_1)
    labels_1 = labels_1[:max_label_size]

    labels_2 = np.where(labels == 2)[0]
    np.random.shuffle(labels_2)
    labels_2 = labels_2[:max_label_size]

    new_data_id = np.hstack((labels_1, labels_2))
    labels = labels[new_data_id]
    sub_id = sub_id[new_data_id]

    return (sub_id, labels)

In [ ]:
sub1,  label1  = balance_dataset(sub_id1, labels1)
sub23, label23 = balance_dataset(sub_id23, labels23)

In [ ]:
data = ['data/nifti/%s.nii.gz' % s for s in sub1]
ds = fmri_dataset(samples=data,
                  targets=label1,
                  chunks=np.zeros(label1.shape),
                  mask='data/mask_gm.nii.gz')
del ds.sa['time_coords']
del ds.sa['time_indices']

h5save('data/mvpa_dataset_scanner1.hdf5', ds)

In [ ]:
data = ['data/nifti/%s.nii.gz' % s for s in sub23]
ds = fmri_dataset(samples=data,
                  targets=label23,
                  chunks=np.zeros(label23.shape),
                  mask='data/mask_gm.nii.gz')
del ds.sa['time_coords']
del ds.sa['time_indices']

h5save('data/mvpa_dataset_scanner23.hdf5', ds)

# Searchlight approach

In [ ]:
import numpy as np
import os
from os.path import join as opj
from mvpa2.base.hdf5 import h5load, h5save
#from mvpa2.clfs.svm import LinearNuSVM
from mvpa2.clfs.gnb import GNB
from mvpa2.clfs.smlr import SMLR
from mvpa2.generators.partition import NFoldPartitioner
from mvpa2.measures.base import CrossValidation
from mvpa2.measures.searchlight import sphere_searchlight
from mvpa2.misc.errorfx import mean_match_accuracy
from mvpa2.mappers.fx import mean_sample
from mvpa2.mappers.zscore import zscore
from mvpa2.suite import map2nifti, time

# Debugger on or off
if __debug__:
    from mvpa2.base import debug
    debug.active = ['SLC']

In [ ]:
###
# 1. Parameters
sphere_radius = 2
nth_element = 100
clfmode = 'SMLR'
clfmode = 'LinearNuSVMC'
clfmode = 'GNB'

cores2use = 8

for postfix in ['_scanner1', '_scanner23']:

    ###
    # 2. Create Dataset
    ds = h5load('data/mvpa_dataset%s.hdf5' % postfix)

    # Specify chunks for the cross validation - Note: This can be done here,
    # because the subjects are well ordered (1/2 controls, 1/2 patients)
    nchunks = 5
    chunks = np.zeros(len(ds))

    for i, e in enumerate(np.array_split(
            range(int(len(ds) / 2.)), nchunks)):
        chunks[e] = i

    for i, e in enumerate(np.array_split(
            range(int(len(ds) / 2.), len(ds)), nchunks)):
        chunks[e] = i

    ds.chunks = chunks

    # Standardize your data
    zscore(ds)

    ###
    # 3. Group Searchlight
    outpath = './results/searchlight'
    if not os.path.exists(outpath):
        os.makedirs(outpath)

    def fill_in_scattered_results(sl, dataset, roi_ids, results):
        """Function to aggregate results - This requires the searchlight
        conditional attribute 'roi_feature_ids' to be enabled"""
        import numpy as np
        from mvpa2.datasets import Dataset
        resmap = None
        for resblock in results:
            for res in resblock:
                if resmap is None:
                    # prepare the result container
                    resmap = np.zeros((len(res), dataset.nfeatures),
                                      dtype=res.samples.dtype)
                    observ_counter = np.zeros(dataset.nfeatures, dtype=int)
                # project the result onto all features -- love broadcasting!
                resmap[:, res.a.roi_feature_ids] += res.samples
                # increment observation counter for all relevant features
                observ_counter[res.a.roi_feature_ids] += 1
        # when all results have been added up average them according to the number
        # of observations
        observ_mask = observ_counter > 0
        resmap[:, observ_mask] /= observ_counter[observ_mask]
        result_ds = Dataset(resmap,
                            fa={'observations': observ_counter})
        if 'mapper' in dataset.a:
            import copy
            result_ds.a['mapper'] = copy.copy(dataset.a.mapper)
        return result_ds

    # specify partitioner
    partitioner = NFoldPartitioner(cvtype=1)

    # Choose right classifier
    if clfmode == 'GNB':
        clf = GNB()

    elif clfmode == 'LinearNuSVMC':
        clf = LinearNuSVMC()

    elif clfmode == 'SMLR':
        smlr_lm = 0.1
        clf = SMLR()


    cv = CrossValidation(clf, partitioner,
                         errorfx=mean_match_accuracy,
                         enable_ca=['stats'])

    sl = sphere_searchlight(cv,
                            radius=sphere_radius,
                            center_ids=range(0,
                                             ds.shape[1],
                                             nth_element),
                            space='voxel_indices',
                            results_fx=fill_in_scattered_results,
                            postproc=mean_sample(),
                            enable_ca=['calling_time', 'roi_feature_ids'])

    # How many cores should be used
    sl.nproc = cores2use

    # Specify identifier
    identifier = '%02dr_%03de_%s_%02dc%s' % (
        sphere_radius, nth_element, clfmode, nchunks, postfix)

    # train classifier on original and permutated dataset
    ofname = opj(outpath, 'sl_clf_%s.hdf5' % identifier)
    sl_map = sl(ds)
    h5save(ofname, sl_map, compression=9)
    print('orig done after %s' % (
        time.strftime(
            '%H:%M:%S',
            time.gmtime(round(sl.ca.calling_time)))))

    # Calculate Classifier Specific outputs
    accuracies = sl_map.samples[0]
    mean_accuracy = accuracies.mean()
    std_accuracy = accuracies.std()
    chance_level = 0.5

    def threshold_above_average(x):
        return chance_level + x * std_accuracy


    def spheres_above_average(x):
        return np.sum(accuracies >= threshold_above_average(x))


    def percent_above_average(x):
        return np.mean(accuracies >= threshold_above_average(x)) * 100


    # Save searchlight accuracy map to NIfTI file
    niftiresults = map2nifti(ds, data=sl_map.S, imghdr=ds.a.imghdr)
    niftiresults.to_filename(opj(outpath, 'nifti_%s.nii.gz' % identifier))

    # Write report to file
    csvfile = opj(outpath, 'report_%s.rst' % identifier)
    with open(csvfile, 'w') as f:
        f.write('Classifier       : {0}\n'.format(clfmode))
        f.write('Postfix          : {0}\n'.format(postfix))
        f.write('Chunks           : {0}\n'.format(nchunks))
        f.write('Sphere Radius    : {0}\n'.format(sphere_radius))
        f.write('N-th Element     : {0}\n'.format(nth_element))
        f.write('Wall Time        : {0}\n'.format(
            time.strftime('%H:%M:%S', time.gmtime(round(sl.ca.calling_time)))))
        f.write('Samples          : {0}\n'.format(ds.S.shape[0]))
        f.write('Features         : {0}\n'.format(ds.S.shape[1]))
        f.write('Volume Dimension : {0}\n'.format(str(ds.a.voxel_dim)))
        f.write('Voxel  Dimension : {0}\n'.format(str(ds.a.voxel_eldim)))
        f.write('CPU              : {0}\n'.format(cores2use))

        f.write('\nChance Level     : {0}\n'.format(round(chance_level, 5)))
        f.write('Accuracy (mean)  : {0:5}\n'.format(round(mean_accuracy, 5)))
        f.write('Accuracy (std)   : {0:5}\n\n'.format(round(std_accuracy, 5)))

        f.write(
            '%Sphere > +2STD  : {0:5}%\n'.format(round(
                percent_above_average(2), 3)))
        f.write(
            '%Sphere > +3STD  : {0:5}%\n'.format(round(
                percent_above_average(3), 3)))
        f.write(
            '%Sphere > +4STD  : {0:5}%\n'.format(round(
                percent_above_average(4), 3)))
        f.write('vSphere > +2STD  : {0:5}\n'.format(spheres_above_average(2)))
        f.write('vSphere > +3STD  : {0:5}\n'.format(spheres_above_average(3)))
        f.write('vSphere > +4STD  : {0:5}\n'.format(spheres_above_average(4)))

        f.write('\n\nDataset Summary:\n')
        f.write('****************\n')
        f.write('%s' % ds.summary())
        f.write('%s' % ds.summary)

    print(accuracies.max())

[SLC] DBG:                        Starting off 8 child processes for nblocks=8
[SLC] DBG:                              Starting computing block for 476 elements
[SLC] DBG:                              Starting computing block for 476 elements
[SLC] DBG:                              Starting computing block for 476 elements
[SLC] DBG:                              Starting computing block for 476 elements
[SLC] DBG:                              Starting computing block for 476 elements
[SLC] DBG:                              +0:00:00 _______[0%]_______ -0:00:33  ROI 47601 (1/476), 33 features[SLC] DBG:                              Starting computing block for 476 elements
[SLC] DBG:                              +0:00:00 _______[0%]_______ -0:00:44  ROI 101 (2/476), 20 featureses[SLC] DBG:                              Starting computing block for 476 elements
[SLC] DBG:                              +0:00:00 _______[0%]_______ -0:00:48  ROI 142801 (1/476), 33 features[SLC] DBG:            